## Data pre-processing

In [1]:
# from google.colab import files
# uploaded = files.upload()

In [2]:
import pandas as pd
df = pd.read_excel('rmi_data.xlsx')
df.head()

,Tender Number,Region,City,Purpose,Publication date,Status,Opening date,Submission deadline,Committee date,Compound Number,...,Winner Name,Final Price,Development Cost,Valuation Price,Minimum Price,Area m2,Plan,Parcel,Block,Section
0,198/2016,חיפה,"זכרון יעקב , מורדות הבאר",בנייה נמוכה/צמודת קרקע,25/09/2016,נדון בוועדת מכרזים,07/11/2016,26/12/2016 12:00,01/01/2017,68,...,"אלוש מישל, אלוש דנה, זלץ בן","2,150,101",NaN,"2,694,000","1,225,770",702,ש/619,68,11308.0,163.0
1,198/2016,חיפה,"זכרון יעקב , מורדות הבאר",בנייה נמוכה/צמודת קרקע,25/09/2016,נדון בוועדת מכרזים,07/11/2016,26/12/2016 12:00,01/01/2017,70,...,"פרץ סלומון, פרץ יפית, גיגי יהודית, גיגי שלמה","2,220,000",NaN,"2,694,000","1,225,770",701,ש/619,70,11308.0,165.0
2,198/2016,חיפה,"זכרון יעקב , מורדות הבאר",בנייה נמוכה/צמודת קרקע,25/09/2016,נדון בוועדת מכרזים,07/11/2016,26/12/2016 12:00,01/01/2017,71,...,"אורגל משה, ברכפלד אביגיל מרים, גלם אפרת, גלם רועי","1,880,800",NaN,"2,694,000","1,225,770",701,ש/619,71,11308.0,166.0
3,197/2016,ירושלים,"קרית גת , כרמי גת",בנייה נמוכה/צמודת קרקע,10/11/2016,נדון בוועדת מכרזים,10/11/2016,19/12/2016 12:00,27/12/2016,15,...,סבן ג'אן איב,"477,770","264,850","239,800","109,109",312,4/140/03/9,15,2909.0,22.0
4,197/2016,ירושלים,"קרית גת , כרמי גת",בנייה נמוכה/צמודת קרקע,10/11/2016,נדון בוועדת מכרזים,10/11/2016,19/12/2016 12:00,27/12/2016,A15,...,סבן ארי,"480,770","264,850","239,800","109,109",312,4/140/03/9,A15,2909.0,23.0


In [3]:
df = df[df['Purpose'] != "בנייה נמוכה/צמודת קרקע -לייזמים/קבלנים"]
df = df.drop(columns=['Tender Number', 'Purpose','Publication date','Status', 'Winner Name', 'Submission deadline', 'Plan','Valuation Price', 'Minimum Price', 'Development Cost', 'Compound Number', 'Opening date'])

In [4]:
df.head()

,Region,City,Committee date,Units,Final Price,Area m2,Parcel,Block,Section
0,חיפה,"זכרון יעקב , מורדות הבאר",01/01/2017,2.0,"2,150,101",702,68,11308.0,163.0
1,חיפה,"זכרון יעקב , מורדות הבאר",01/01/2017,2.0,"2,220,000",701,70,11308.0,165.0
2,חיפה,"זכרון יעקב , מורדות הבאר",01/01/2017,2.0,"1,880,800",701,71,11308.0,166.0
3,ירושלים,"קרית גת , כרמי גת",27/12/2016,1.0,"477,770",312,15,2909.0,22.0
4,ירושלים,"קרית גת , כרמי גת",27/12/2016,1.0,"480,770",312,A15,2909.0,23.0


### Cleaning data
#### Using Min-Max Scaler

In [5]:
# Remove missing values

print("missing data:")
print(df.isnull().sum())

df = df[df['Final Price'].notnull()]
df = df[df['Region'].notnull()]
df = df[df['Final Price'] != "אין הצעות למתחם זה"]

numeric_columns = ['Final Price']

for col in numeric_columns:
    df[col] = df[col].astype(str).str.replace(',', '').astype(float)

# Handling missing values in numerics columns
df['Units'] = df['Units'].fillna(df['Units'].median())

df['Parcel'] = df['Parcel'].fillna("Not existing")
df['Block'] = df['Block'].fillna("Not existing")
df['Section'] = df['Section'].fillna("Not existing") # remove this
#
# df['Block'] = pd.to_numeric(df['Block'], errors='coerce')
# df['Section'] = pd.to_numeric(df['Block'], errors='coerce')
# df['Parcel'] = pd.to_numeric(df['Parcel'], errors='coerce')

missing data:
Region             396
City               396
Committee date     396
Units              227
Final Price       3480
Area m2              2
Parcel             610
Block              259
Section            259
dtype: int64


In [6]:
df['Final Price'] = df['Final Price'].replace(',', '', regex=True).astype(float)
df['Area m2'] = df['Area m2'].replace(',', '', regex=True).astype(float)
print(f"before filter Final Price: {len(df)}")

# Manual binderies
lower_bound = df['Final Price'].quantile(0.05)
upper_bound = df['Final Price'].quantile(0.95)

df = df[(df['Final Price'] >= lower_bound) & (df['Final Price'] <= upper_bound)]
print(f"after filter Final Price: {len(df)}")
df = df[(df['Area m2'] >= 100) & (df['Area m2'] <= 1000)]
print(f"after filter Area m2: {len(df)}")

before filter Final Price: 7484
after filter Final Price: 6735
after filter Area m2: 6467


In [7]:
df['Committee date'] = pd.to_datetime(df['Committee date'], errors='coerce')
df['Committee_year'] = df['Committee date'].dt.year
df['Committee_month'] = df['Committee date'].dt.month

df = df.drop(columns=['Committee date'])

In [8]:
print(df.head())

    Region                      City  Units  Final Price  Area m2 Parcel  \
0     חיפה  זכרון יעקב , מורדות הבאר    2.0    2150101.0    702.0     68   
1     חיפה  זכרון יעקב , מורדות הבאר    2.0    2220000.0    701.0     70   
2     חיפה  זכרון יעקב , מורדות הבאר    2.0    1880800.0    701.0     71   
3  ירושלים         קרית גת , כרמי גת    1.0     477770.0    312.0     15   
4  ירושלים         קרית גת , כרמי גת    1.0     480770.0    312.0    A15   

     Block Section  Committee_year  Committee_month  
0  11308.0   163.0          2017.0              1.0  
1  11308.0   165.0          2017.0              1.0  
2  11308.0   166.0          2017.0              1.0  
3   2909.0    22.0             NaN              NaN  
4   2909.0    23.0             NaN              NaN  


In [9]:
print(df['Final Price'].describe())

count    6.467000e+03
mean     5.728593e+05
std      4.968573e+05
min      3.636000e+04
25%      2.213245e+05
50%      4.300000e+05
75%      7.577635e+05
max      2.455560e+06
Name: Final Price, dtype: float64


In [10]:
df['Price per m2'] = df['Final Price'] / df['Area m2']

# Saving min- max vals of price.
price_per_m2_min = df['Price per m2'].min()
price_per_m2_max = df['Price per m2'].max()

In [11]:
import json
price_minmax = {
    "min": float(price_per_m2_min),
    "max": float(price_per_m2_max)
}

with open("price_minmax.json", "w", encoding="utf-8") as f:
    json.dump(price_minmax, f, ensure_ascii=False, indent=2)

In [12]:
numeric_columns = ['Units','Final Price','Area m2','Price per m2']

scaler_dict = {}

for col in numeric_columns:
    if col not in df.columns:
        print(f"Warning: Column '{col}' not found, skipping.")
        continue

    col_min = df[col].min()
    col_max = df[col].max()
    if col_min == col_max:
        print(f"Warning: column '{col}' has the same min ({col_min}) and max. Setting all scaled to 0.")
        df[col] = 0.0
    else:
        df[col] = (df[col] - col_min) / (col_max - col_min)

    scaler_dict[col] = {
        "min": float(col_min),
        "max": float(col_max)
    }

block_one_hot = pd.get_dummies(df['Block'], prefix='Block')
# section_one_hot = pd.get_dummies(df['Section'], prefix='Section')
# parcel_one_hot = pd.get_dummies(df['Parcel'], prefix='Parcel')
committee_month_one_hot = pd.get_dummies(df['Committee_month'], prefix='Committee_month')
committee_year_one_hot = pd.get_dummies(df['Committee_year'], prefix='Committee_year')


df = df.drop(['Block', 'Section', 'Parcel', 'Committee_month', 'Committee_year'], axis=1)

# df = pd.concat([df, block_one_hot, section_one_hot, parcel_one_hot, committee_month_one_hot, committee_year_one_hot], axis=1)
df = pd.concat([df, block_one_hot, committee_month_one_hot, committee_year_one_hot], axis=1)

with open("scaler_minmax.json", "w", encoding="utf-8") as f:
    json.dump(scaler_dict, f, ensure_ascii=False, indent=2)

normalized_columns = ['Final Price', 'Area m2', 'Units']

In [13]:
df.head()

,Region,City,Units,Final Price,Area m2,Price per m2,Block_325.0,Block_326.0,Block_375.0,Block_376.0,...,Committee_year_2016.0,Committee_year_2017.0,Committee_year_2018.0,Committee_year_2019.0,Committee_year_2020.0,Committee_year_2021.0,Committee_year_2022.0,Committee_year_2023.0,Committee_year_2024.0,Committee_year_2025.0
0,חיפה,"זכרון יעקב , מורדות הבאר",0.2,0.873736,0.639225,0.249075,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,חיפה,"זכרון יעקב , מורדות הבאר",0.2,0.902629,0.638015,0.257707,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
2,חיפה,"זכרון יעקב , מורדות הבאר",0.2,0.762417,0.638015,0.217580,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
3,ירושלים,"קרית גת , כרמי גת",0.0,0.182461,0.167070,0.122072,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,ירושלים,"קרית גת , כרמי גת",0.0,0.183701,0.167070,0.122869,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [14]:
df = pd.get_dummies(df, columns=['Region', 'City'], prefix=['Region', 'City']) # קידוד 0-1 לערים ומחוזות

In [15]:
city_columns = [col for col in df.columns if col.startswith('City_')]

cities = set()
for col in city_columns:
    city_name = col.replace('City_', '').split(',')[0].strip()
    cities.add(city_name)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6467 entries, 0 to 11491
Columns: 536 entries, Units to City_תנובות
dtypes: bool(532), float64(4)
memory usage: 3.5 MB


In [17]:
df.columns

Index(['Units', 'Final Price', 'Area m2', 'Price per m2', 'Block_325.0',
       'Block_326.0', 'Block_375.0', 'Block_376.0', 'Block_390.0',
       'Block_395.0',
       ...
       'City_שלומי , יערית', 'City_שער אפרים', 'City_שער אפרים , שער אפרים',
       'City_שריגים(לי און', 'City_שריגים(לי און , מועצה אזורית מטה יהודה',
       'City_תל אביב יפו , בצרה', 'City_תל מונד', 'City_תלמי יחיאל',
       'City_תמרת', 'City_תנובות'],
      dtype='object', length=536)

## Focus on Southern cities of Israel

In [18]:
# southern_base_cities = [
#     "אופקים", "אשקלון", "באר גנים", "באר שבע", "גדרה", "גן יבנה",
#     "חלץ", "יבנה", "מיתר", "להבים", "דימונה", "ניצן", "נתיבות",
#     "עומר", "קרית גת", "קרית מלאכי", "שדרות"
# ]
#
# southern_city_columns = []
# for city in southern_base_cities:
#     matches = [col for col in df.columns if col.startswith(f"City_{city}")]
#     southern_city_columns.extend(matches)
#
# df = df[df[southern_city_columns].any(axis=1)]
#
# all_city_columns = [col for col in df.columns if col.startswith("City_")]
# columns_with_data = [col for col in all_city_columns if df[col].sum() > 0]
#
# columns_to_drop = set(all_city_columns) - set(columns_with_data)
# df = df.drop(columns=columns_to_drop)
#
# print(columns_with_data)

In [19]:
df.head()

,Units,Final Price,Area m2,Price per m2,Block_325.0,Block_326.0,Block_375.0,Block_376.0,Block_390.0,Block_395.0,...,"City_שלומי , יערית",City_שער אפרים,"City_שער אפרים , שער אפרים",City_שריגים(לי און,"City_שריגים(לי און , מועצה אזורית מטה יהודה","City_תל אביב יפו , בצרה",City_תל מונד,City_תלמי יחיאל,City_תמרת,City_תנובות
0,0.2,0.873736,0.639225,0.249075,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,0.2,0.902629,0.638015,0.257707,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,0.2,0.762417,0.638015,0.217580,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,0.0,0.182461,0.167070,0.122072,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,0.0,0.183701,0.167070,0.122869,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [20]:
df=df.drop(['Final Price'],axis=1)

In [21]:
df.to_csv("cleaned_data.csv", index=False)

# files.download("/content/cleaned_data.csv")
# files.download("/content/scaler_minmax.json")

In [22]:
df.columns

Index(['Units', 'Area m2', 'Price per m2', 'Block_325.0', 'Block_326.0',
       'Block_375.0', 'Block_376.0', 'Block_390.0', 'Block_395.0',
       'Block_399.0',
       ...
       'City_שלומי , יערית', 'City_שער אפרים', 'City_שער אפרים , שער אפרים',
       'City_שריגים(לי און', 'City_שריגים(לי און , מועצה אזורית מטה יהודה',
       'City_תל אביב יפו , בצרה', 'City_תל מונד', 'City_תלמי יחיאל',
       'City_תמרת', 'City_תנובות'],
      dtype='object', length=535)